<a href="https://colab.research.google.com/github/Wayne122/CMPE257/blob/main/Alternus_Vera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alternus Vera

Topic: Content Statistics

Author: Han-Wei Lin

##Import library and data

### Liar Liar dataset

Description of the TSV format:

Column 1: the ID of the statement ([ID].json).

Column 2: the label.

Column 3: the statement.

Column 4: the subject(s).

Column 5: the speaker.

Column 6: the speaker's job title.

Column 7: the state info.

Column 8: the party affiliation.

Column 9-13: the total credit history count, including the current statement.

9: barely true counts.

10: false counts.

11: half true counts.

12: mostly true counts.

13: pants on fire counts.

Column 14: the context (venue / location of the speech or statement).

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1y3yYF5HHPhH7SyaPwPU9H5HApJzevsFK"})
downloaded.GetContentFile('liar_dataset.zip')

In [3]:
from zipfile import ZipFile

In [4]:
with ZipFile('liar_dataset.zip', 'r') as myzip:
    train_data = myzip.open('train.tsv')
    test_data = myzip.open('test.tsv')
    valid_data = myzip.open('valid.tsv')
    readme = myzip.open('README')

train_df = pd.read_csv(train_data, sep='\t', header=None)
test_df = pd.read_csv(test_data, sep='\t', header=None)
valid_df = pd.read_csv(valid_data, sep='\t', header=None)

for i in readme.read().splitlines():
    print(i.decode('utf-8'))

LIAR: A BENCHMARK DATASET FOR FAKE NEWS DETECTION

William Yang Wang, "Liar, Liar Pants on Fire": A New Benchmark Dataset for Fake News Detection, to appear in Proceedings of the 55th Annual Meeting of the Association for Computational Linguistics (ACL 2017), short paper, Vancouver, BC, Canada, July 30-August 4, ACL.
Description of the TSV format:

Column 1: the ID of the statement ([ID].json).
Column 2: the label.
Column 3: the statement.
Column 4: the subject(s).
Column 5: the speaker.
Column 6: the speaker's job title.
Column 7: the state info.
Column 8: the party affiliation.
Column 9-13: the total credit history count, including the current statement.
9: barely true counts.
10: false counts.
11: half true counts.
12: mostly true counts.
13: pants on fire counts.
Column 14: the context (venue / location of the speech or statement).

Note that we do not provide the full-text verdict report in this current version of the dataset,
but you can use the following command to access the fu

In [ ]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [ ]:
train_df.describe()

,8,9,10,11,12
count,10238.000000,10238.000000,10238.000000,10238.000000,10238.000000
mean,11.533210,13.286482,17.133718,16.434265,6.201407
std,18.973764,24.112936,35.846511,36.151688,16.128927
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,2.000000,3.000000,3.000000,1.000000
75%,12.000000,12.000000,13.000000,11.000000,5.000000
max,70.000000,114.000000,160.000000,163.000000,105.000000


### Fake and real news dataset Kaggle

https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset


In [ ]:
with ZipFile('archive.zip', 'r') as myzip:
    fake_data = myzip.open('Fake.csv')
    true_data = myzip.open('True.csv')

fake_df = pd.read_csv(fake_data)
true_df = pd.read_csv(true_data)
fake_df.head()
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


### Kaggle fake news dataset

https://www.kaggle.com/c/fake-news/data


In [ ]:
with ZipFile('fake-news.zip', 'r') as myzip:
    train_data = myzip.open('train.csv')
    test_data = myzip.open('test.csv')
    label_data = myzip.open('submit.csv')

train_df2 = pd.read_csv(train_data)
test_df2 = pd.read_csv(test_data)
label_df = pd.read_csv(label_data)
train_df2.describe()
test_df2.describe()
label_df.describe() # label of test data

,id,label
count,5200.000000,5200.000000
mean,23399.500000,0.550192
std,1501.255031,0.497522
min,20800.000000,0.000000
25%,22099.750000,0.000000
50%,23399.500000,1.000000
75%,24699.250000,1.000000
max,25999.000000,1.000000


## Liar Liar dataset

### Data preparation (POS counter)

#### Download tools

In [5]:
import nltk
nltk.download('punkt') # word_tokenize
nltk.download('averaged_perceptron_tagger') # pos_tag
nltk.download('tagsets') # help.upenn_tagset() to list all possible tags

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

#### All possible pos tags

In [6]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

#### Train data

##### Transforming token to pos tag

In [7]:
train_postags = []
for i, c in enumerate(train_df[2]):
    train_postags.append(nltk.pos_tag(nltk.word_tokenize(c)))
    if i < 5:
        print(train_postags[-1])

[('Says', 'VBZ'), ('the', 'DT'), ('Annies', 'NNPS'), ('List', 'NNP'), ('political', 'JJ'), ('group', 'NN'), ('supports', 'VBZ'), ('third-trimester', 'NN'), ('abortions', 'NNS'), ('on', 'IN'), ('demand', 'NN'), ('.', '.')]
[('When', 'WRB'), ('did', 'VBD'), ('the', 'DT'), ('decline', 'NN'), ('of', 'IN'), ('coal', 'NN'), ('start', 'NN'), ('?', '.'), ('It', 'PRP'), ('started', 'VBD'), ('when', 'WRB'), ('natural', 'JJ'), ('gas', 'NN'), ('took', 'VBD'), ('off', 'RP'), ('that', 'IN'), ('started', 'VBD'), ('to', 'TO'), ('begin', 'VB'), ('in', 'IN'), ('(', '('), ('President', 'NNP'), ('George', 'NNP'), ('W.', 'NNP'), (')', ')'), ('Bushs', 'NNP'), ('administration', 'NN'), ('.', '.')]
[('Hillary', 'NNP'), ('Clinton', 'NNP'), ('agrees', 'VBZ'), ('with', 'IN'), ('John', 'NNP'), ('McCain', 'NNP'), ('``', '``'), ('by', 'IN'), ('voting', 'VBG'), ('to', 'TO'), ('give', 'VB'), ('George', 'NNP'), ('Bush', 'NNP'), ('the', 'DT'), ('benefit', 'NN'), ('of', 'IN'), ('the', 'DT'), ('doubt', 'NN'), ('on', 'IN'

##### Pos count

In [8]:
train_tag_counters = []

for tags in train_postags:
    tag_counter = {
        '$': 0,
        "''": 0,
        '(': 0,
        ')': 0,
        ',': 0,
        '--': 0,
        '.': 0,
        ':': 0,
        'CC': 0,
        'CD': 0,
        'DT': 0,
        'EX': 0,
        'FW': 0,
        'IN': 0,
        'JJ': 0,
        'JJR': 0,
        'JJS': 0,
        'LS': 0,
        'MD': 0,
        'NN': 0,
        'NNP': 0,
        'NNPS': 0,
        'NNS': 0,
        'PDT': 0,
        'POS': 0,
        'PRP': 0,
        'PRP$': 0,
        'RB': 0,
        'RBR': 0,
        'RBS': 0,
        'RP': 0,
        'SYM': 0,
        'TO': 0,
        'UH': 0,
        'VB': 0,
        'VBD': 0,
        'VBG': 0,
        'VBN': 0,
        'VBP': 0,
        'VBZ': 0,
        'WDT': 0,
        'WP': 0,
        'WP$': 0,
        'WRB': 0,
        '``': 0,
        '#': 0
    }
    for tag in tags:
        if tag[1] in tag_counter:
            tag_counter[tag[1]] += 1
        else:
            print(tag)
    train_tag_counters.append(tag_counter)

##### Create dataframe with POS count

In [9]:
dl = []
cqml = []
opl = []
cpl = []
cl = []
dal = []
stl = []
cel = []
ccl = []
cdl = []
dtl = []
exl = []
fwl = []
inl = []
jjl = []
jjrl = []
jjsl = []
lsl = []
mdl = []
nnl = []
nnpl = []
nnpsl = []
nnsl = []
pdtl = []
posl = []
prpl = []
prpdl = []
rbl = []
rbrl = []
rbsl = []
rpl = []
syml = []
tol = []
uhl = []
vbl = []
vbdl = []
vbgl = []
vbnl = []
vbpl = []
vbzl = []
wdtl = []
wpl = []
wpdl = []
wrbl = []
oqml = []
pl = []

for tct in train_tag_counters:
    tct_values = list(tct.values())
    dl.append(tct_values[0])
    cqml.append(tct_values[1])
    opl.append(tct_values[2])
    cpl.append(tct_values[3])
    cl.append(tct_values[4])
    dal.append(tct_values[5])
    stl.append(tct_values[6])
    cel.append(tct_values[7])
    ccl.append(tct_values[8])
    cdl.append(tct_values[9])
    dtl.append(tct_values[10])
    exl.append(tct_values[11])
    fwl.append(tct_values[12])
    inl.append(tct_values[13])
    jjl.append(tct_values[14])
    jjrl.append(tct_values[15])
    jjsl.append(tct_values[16])
    lsl.append(tct_values[17])
    mdl.append(tct_values[18])
    nnl.append(tct_values[19])
    nnpl.append(tct_values[20])
    nnpsl.append(tct_values[21])
    nnsl.append(tct_values[22])
    pdtl.append(tct_values[23])
    posl.append(tct_values[24])
    prpl.append(tct_values[25])
    prpdl.append(tct_values[26])
    rbl.append(tct_values[27])
    rbrl.append(tct_values[28])
    rbsl.append(tct_values[29])
    rpl.append(tct_values[30])
    syml.append(tct_values[31])
    tol.append(tct_values[32])
    uhl.append(tct_values[33])
    vbl.append(tct_values[34])
    vbdl.append(tct_values[35])
    vbgl.append(tct_values[36])
    vbnl.append(tct_values[37])
    vbpl.append(tct_values[38])
    vbzl.append(tct_values[39])
    wdtl.append(tct_values[40])
    wpl.append(tct_values[41])
    wpdl.append(tct_values[42])
    wrbl.append(tct_values[43])
    oqml.append(tct_values[44])
    pl.append(tct_values[45])

In [10]:
#train_df_prepro = pd.get_dummies(train_df[1], columns=1)
train_df_prepro = train_df.loc[:, [1]]

In [11]:
train_df_prepro['$'] = dl
train_df_prepro["''"] = cqml
train_df_prepro['('] = opl
train_df_prepro[')'] = cpl
train_df_prepro[','] = cl
train_df_prepro['--'] = dal
train_df_prepro['.'] = stl
train_df_prepro[':'] = cel
train_df_prepro['CC'] = ccl
train_df_prepro['CD'] = cdl
train_df_prepro['DT'] = dtl
train_df_prepro['EX'] = exl
train_df_prepro['FW'] = fwl
train_df_prepro['IN'] = inl
train_df_prepro['JJ'] = jjl
train_df_prepro['JJR'] = jjrl
train_df_prepro['JJS'] = jjsl
train_df_prepro['LS'] = lsl
train_df_prepro['MD'] = mdl
train_df_prepro['NN'] = nnl
train_df_prepro['NNP'] = nnpl
train_df_prepro['NNPS'] = nnpsl
train_df_prepro['NNS'] = nnsl
train_df_prepro['PDT'] = pdtl
train_df_prepro['POS'] = posl
train_df_prepro['PRP'] = prpl
train_df_prepro['PRP$'] = prpdl
train_df_prepro['RB'] = rbl
train_df_prepro['RBR'] = rbrl
train_df_prepro['RBS'] = rbsl
train_df_prepro['RP'] = rpl
train_df_prepro['SYM'] = syml
train_df_prepro['TO'] = tol
train_df_prepro['UH'] = uhl
train_df_prepro['VB'] = vbl
train_df_prepro['VBD'] = vbdl
train_df_prepro['VBG'] = vbgl
train_df_prepro['VBN'] = vbnl
train_df_prepro['VBZ'] = vbzl
train_df_prepro['WDT'] = wdtl
train_df_prepro['WP'] = wpl
train_df_prepro['WP$'] = wpdl
train_df_prepro['WRB'] = wrbl
train_df_prepro['``'] = oqml
train_df_prepro['#'] = pl

train_df_prepro

,1,$,'',(,),",",--,.,:,CC,CD,DT,EX,FW,IN,JJ,JJR,JJS,LS,MD,NN,NNP,NNPS,NNS,PDT,POS,PRP,PRP$,RB,RBR,RBS,RP,SYM,TO,UH,VB,VBD,VBG,VBN,VBZ,WDT,WP,WP$,WRB,``,#
0,false,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0
1,half-true,0,0,1,1,0,0,2,0,0,0,1,0,0,3,1,0,0,0,0,5,4,0,0,0,0,1,0,0,0,0,1,0,1,0,1,4,0,0,0,0,0,0,2,0,0
2,mostly-true,0,1,0,0,0,0,1,0,0,0,2,0,0,4,0,0,0,0,0,2,7,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0
3,false,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,5,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0
4,half-true,0,0,0,0,0,0,1,0,0,0,2,0,0,2,1,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,mostly-true,0,0,0,0,0,0,1,0,0,0,1,2,0,4,0,1,0,0,0,4,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10236,mostly-true,0,0,0,0,0,0,1,0,1,0,2,0,0,2,0,0,0,0,0,2,4,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
10237,half-true,0,0,0,0,2,0,1,0,0,0,2,0,0,5,1,1,0,0,1,3,7,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,3,1,0,0,0,0,0
10238,false,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0


In [12]:
#target_labels = ['barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'true']
#X_train = train_df_prepro.drop(target_labels, axis=1)
#y_train = train_df_prepro[target_labels]

X_train = train_df_prepro.drop(1, axis=1)
y_train = train_df_prepro[1]

#### Valid data

##### Transforming token to pos tag

In [82]:
val_postags = []
for i, c in enumerate(valid_df[2]):
    val_postags.append(nltk.pos_tag(nltk.word_tokenize(c)))
    if i < 5:
        print(val_postags[-1])

[('We', 'PRP'), ('have', 'VBP'), ('less', 'JJR'), ('Americans', 'NNPS'), ('working', 'VBG'), ('now', 'RB'), ('than', 'IN'), ('in', 'IN'), ('the', 'DT'), ('70s', 'CD'), ('.', '.')]
[('When', 'WRB'), ('Obama', 'NNP'), ('was', 'VBD'), ('sworn', 'VBN'), ('into', 'IN'), ('office', 'NN'), (',', ','), ('he', 'PRP'), ('DID', 'VBZ'), ('NOT', 'NNP'), ('use', 'VB'), ('the', 'DT'), ('Holy', 'NNP'), ('Bible', 'NNP'), (',', ','), ('but', 'CC'), ('instead', 'RB'), ('the', 'DT'), ('Kuran', 'NNP'), ('(', '('), ('Their', 'PRP$'), ('equivalency', 'NN'), ('to', 'TO'), ('our', 'PRP$'), ('Bible', 'JJ'), (',', ','), ('but', 'CC'), ('very', 'RB'), ('different', 'JJ'), ('beliefs', 'NNS'), (')', ')'), ('.', '.')]
[('Says', 'VBZ'), ('Having', 'VBG'), ('organizations', 'NNS'), ('parading', 'VBG'), ('as', 'IN'), ('being', 'VBG'), ('social', 'JJ'), ('welfare', 'NN'), ('organizations', 'NNS'), ('and', 'CC'), ('then', 'RB'), ('being', 'VBG'), ('involved', 'VBN'), ('in', 'IN'), ('the', 'DT'), ('political', 'JJ'), ('co

##### Pos count

In [83]:
val_tag_counters = []

for tags in val_postags:
    tag_counter = {
        '$': 0,
        "''": 0,
        '(': 0,
        ')': 0,
        ',': 0,
        '--': 0,
        '.': 0,
        ':': 0,
        'CC': 0,
        'CD': 0,
        'DT': 0,
        'EX': 0,
        'FW': 0,
        'IN': 0,
        'JJ': 0,
        'JJR': 0,
        'JJS': 0,
        'LS': 0,
        'MD': 0,
        'NN': 0,
        'NNP': 0,
        'NNPS': 0,
        'NNS': 0,
        'PDT': 0,
        'POS': 0,
        'PRP': 0,
        'PRP$': 0,
        'RB': 0,
        'RBR': 0,
        'RBS': 0,
        'RP': 0,
        'SYM': 0,
        'TO': 0,
        'UH': 0,
        'VB': 0,
        'VBD': 0,
        'VBG': 0,
        'VBN': 0,
        'VBP': 0,
        'VBZ': 0,
        'WDT': 0,
        'WP': 0,
        'WP$': 0,
        'WRB': 0,
        '``': 0,
        '#': 0
    }
    for tag in tags:
        if tag[1] in tag_counter:
            tag_counter[tag[1]] += 1
        else:
            print(tag)
    val_tag_counters.append(tag_counter)

##### Create dataframe with POS count

In [84]:
dl = []
cqml = []
opl = []
cpl = []
cl = []
dal = []
stl = []
cel = []
ccl = []
cdl = []
dtl = []
exl = []
fwl = []
inl = []
jjl = []
jjrl = []
jjsl = []
lsl = []
mdl = []
nnl = []
nnpl = []
nnpsl = []
nnsl = []
pdtl = []
posl = []
prpl = []
prpdl = []
rbl = []
rbrl = []
rbsl = []
rpl = []
syml = []
tol = []
uhl = []
vbl = []
vbdl = []
vbgl = []
vbnl = []
vbpl = []
vbzl = []
wdtl = []
wpl = []
wpdl = []
wrbl = []
oqml = []
pl = []

for tct in val_tag_counters:
    tct_values = list(tct.values())
    dl.append(tct_values[0])
    cqml.append(tct_values[1])
    opl.append(tct_values[2])
    cpl.append(tct_values[3])
    cl.append(tct_values[4])
    dal.append(tct_values[5])
    stl.append(tct_values[6])
    cel.append(tct_values[7])
    ccl.append(tct_values[8])
    cdl.append(tct_values[9])
    dtl.append(tct_values[10])
    exl.append(tct_values[11])
    fwl.append(tct_values[12])
    inl.append(tct_values[13])
    jjl.append(tct_values[14])
    jjrl.append(tct_values[15])
    jjsl.append(tct_values[16])
    lsl.append(tct_values[17])
    mdl.append(tct_values[18])
    nnl.append(tct_values[19])
    nnpl.append(tct_values[20])
    nnpsl.append(tct_values[21])
    nnsl.append(tct_values[22])
    pdtl.append(tct_values[23])
    posl.append(tct_values[24])
    prpl.append(tct_values[25])
    prpdl.append(tct_values[26])
    rbl.append(tct_values[27])
    rbrl.append(tct_values[28])
    rbsl.append(tct_values[29])
    rpl.append(tct_values[30])
    syml.append(tct_values[31])
    tol.append(tct_values[32])
    uhl.append(tct_values[33])
    vbl.append(tct_values[34])
    vbdl.append(tct_values[35])
    vbgl.append(tct_values[36])
    vbnl.append(tct_values[37])
    vbpl.append(tct_values[38])
    vbzl.append(tct_values[39])
    wdtl.append(tct_values[40])
    wpl.append(tct_values[41])
    wpdl.append(tct_values[42])
    wrbl.append(tct_values[43])
    oqml.append(tct_values[44])
    pl.append(tct_values[45])

In [85]:
#val_df_prepro = pd.get_dummies(valid_df[1], columns=1)
val_df_prepro = valid_df.loc[:, [1]]

In [86]:
val_df_prepro['$'] = dl
val_df_prepro["''"] = cqml
val_df_prepro['('] = opl
val_df_prepro[')'] = cpl
val_df_prepro[','] = cl
val_df_prepro['--'] = dal
val_df_prepro['.'] = stl
val_df_prepro[':'] = cel
val_df_prepro['CC'] = ccl
val_df_prepro['CD'] = cdl
val_df_prepro['DT'] = dtl
val_df_prepro['EX'] = exl
val_df_prepro['FW'] = fwl
val_df_prepro['IN'] = inl
val_df_prepro['JJ'] = jjl
val_df_prepro['JJR'] = jjrl
val_df_prepro['JJS'] = jjsl
val_df_prepro['LS'] = lsl
val_df_prepro['MD'] = mdl
val_df_prepro['NN'] = nnl
val_df_prepro['NNP'] = nnpl
val_df_prepro['NNPS'] = nnpsl
val_df_prepro['NNS'] = nnsl
val_df_prepro['PDT'] = pdtl
val_df_prepro['POS'] = posl
val_df_prepro['PRP'] = prpl
val_df_prepro['PRP$'] = prpdl
val_df_prepro['RB'] = rbl
val_df_prepro['RBR'] = rbrl
val_df_prepro['RBS'] = rbsl
val_df_prepro['RP'] = rpl
val_df_prepro['SYM'] = syml
val_df_prepro['TO'] = tol
val_df_prepro['UH'] = uhl
val_df_prepro['VB'] = vbl
val_df_prepro['VBD'] = vbdl
val_df_prepro['VBG'] = vbgl
val_df_prepro['VBN'] = vbnl
val_df_prepro['VBZ'] = vbzl
val_df_prepro['WDT'] = wdtl
val_df_prepro['WP'] = wpl
val_df_prepro['WP$'] = wpdl
val_df_prepro['WRB'] = wrbl
val_df_prepro['``'] = oqml
val_df_prepro['#'] = pl

val_df_prepro

,1,$,'',(,),",",--,.,:,CC,CD,DT,EX,FW,IN,JJ,JJR,JJS,LS,MD,NN,NNP,NNPS,NNS,PDT,POS,PRP,PRP$,RB,RBR,RBS,RP,SYM,TO,UH,VB,VBD,VBG,VBN,VBZ,WDT,WP,WP$,WRB,``,#
0,barely-true,0,0,0,0,0,0,1,0,0,1,1,0,0,2,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,pants-fire,0,0,1,1,3,0,1,0,2,0,2,0,0,1,2,0,0,0,0,2,5,0,1,0,0,1,2,2,0,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0
2,false,0,0,0,0,0,0,1,0,1,1,3,0,0,3,2,0,0,0,0,3,0,0,3,0,0,1,0,3,0,0,0,0,1,0,0,2,4,2,2,0,0,0,1,0,0
3,half-true,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,half-true,0,1,0,0,2,0,1,0,0,1,3,0,0,8,4,1,0,0,0,4,0,1,2,1,0,2,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,half-true,0,0,1,1,1,0,1,0,0,0,3,0,0,6,1,2,0,0,0,4,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
1280,mostly-true,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,1,4,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0
1281,true,0,1,0,0,0,0,1,0,1,0,1,0,0,1,1,0,0,0,0,3,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1282,false,0,0,0,0,0,0,2,1,1,1,5,0,0,1,3,0,0,0,0,11,0,0,0,0,1,2,0,0,0,0,0,0,1,0,1,0,2,0,2,0,0,0,0,0,0


In [87]:
#target_labels = ['barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'true']
#X_val = val_df_prepro.drop(target_labels, axis=1)
#y_val = val_df_prepro[target_labels]

X_val = val_df_prepro.drop(1, axis=1)
y_val = val_df_prepro[1]

#### Test data

##### Transforming token to pos tag

In [13]:
test_postags = []
for i, c in enumerate(test_df[2]):
    test_postags.append(nltk.pos_tag(nltk.word_tokenize(c)))
    if i < 5:
        print(test_postags[-1])

[('Building', 'VBG'), ('a', 'DT'), ('wall', 'NN'), ('on', 'IN'), ('the', 'DT'), ('U.S.-Mexico', 'NNP'), ('border', 'NN'), ('will', 'MD'), ('take', 'VB'), ('literally', 'RB'), ('years', 'NNS'), ('.', '.')]
[('Wisconsin', 'NNP'), ('is', 'VBZ'), ('on', 'IN'), ('pace', 'NN'), ('to', 'TO'), ('double', 'VB'), ('the', 'DT'), ('number', 'NN'), ('of', 'IN'), ('layoffs', 'NNS'), ('this', 'DT'), ('year', 'NN'), ('.', '.')]
[('Says', 'VBZ'), ('John', 'NNP'), ('McCain', 'NNP'), ('has', 'VBZ'), ('done', 'VBN'), ('nothing', 'NN'), ('to', 'TO'), ('help', 'VB'), ('the', 'DT'), ('vets', 'NNS'), ('.', '.')]
[('Suzanne', 'NNP'), ('Bonamici', 'NNP'), ('supports', 'VBZ'), ('a', 'DT'), ('plan', 'NN'), ('that', 'WDT'), ('will', 'MD'), ('cut', 'VB'), ('choice', 'NN'), ('for', 'IN'), ('Medicare', 'NNP'), ('Advantage', 'NNP'), ('seniors', 'NNS'), ('.', '.')]
[('When', 'WRB'), ('asked', 'VBN'), ('by', 'IN'), ('a', 'DT'), ('reporter', 'NN'), ('whether', 'IN'), ('hes', 'NNS'), ('at', 'IN'), ('the', 'DT'), ('center'

##### Pos count

In [14]:
test_tag_counters = []

for tags in test_postags:
    tag_counter = {
        '$': 0,
        "''": 0,
        '(': 0,
        ')': 0,
        ',': 0,
        '--': 0,
        '.': 0,
        ':': 0,
        'CC': 0,
        'CD': 0,
        'DT': 0,
        'EX': 0,
        'FW': 0,
        'IN': 0,
        'JJ': 0,
        'JJR': 0,
        'JJS': 0,
        'LS': 0,
        'MD': 0,
        'NN': 0,
        'NNP': 0,
        'NNPS': 0,
        'NNS': 0,
        'PDT': 0,
        'POS': 0,
        'PRP': 0,
        'PRP$': 0,
        'RB': 0,
        'RBR': 0,
        'RBS': 0,
        'RP': 0,
        'SYM': 0,
        'TO': 0,
        'UH': 0,
        'VB': 0,
        'VBD': 0,
        'VBG': 0,
        'VBN': 0,
        'VBP': 0,
        'VBZ': 0,
        'WDT': 0,
        'WP': 0,
        'WP$': 0,
        'WRB': 0,
        '``': 0,
        '#': 0
    }
    for tag in tags:
        if tag[1] in tag_counter:
            tag_counter[tag[1]] += 1
        else:
            print(tag)
    test_tag_counters.append(tag_counter)

##### Create dataframe with POS count

In [15]:
dl = []
cqml = []
opl = []
cpl = []
cl = []
dal = []
stl = []
cel = []
ccl = []
cdl = []
dtl = []
exl = []
fwl = []
inl = []
jjl = []
jjrl = []
jjsl = []
lsl = []
mdl = []
nnl = []
nnpl = []
nnpsl = []
nnsl = []
pdtl = []
posl = []
prpl = []
prpdl = []
rbl = []
rbrl = []
rbsl = []
rpl = []
syml = []
tol = []
uhl = []
vbl = []
vbdl = []
vbgl = []
vbnl = []
vbpl = []
vbzl = []
wdtl = []
wpl = []
wpdl = []
wrbl = []
oqml = []
pl = []

for tct in test_tag_counters:
    tct_values = list(tct.values())
    dl.append(tct_values[0])
    cqml.append(tct_values[1])
    opl.append(tct_values[2])
    cpl.append(tct_values[3])
    cl.append(tct_values[4])
    dal.append(tct_values[5])
    stl.append(tct_values[6])
    cel.append(tct_values[7])
    ccl.append(tct_values[8])
    cdl.append(tct_values[9])
    dtl.append(tct_values[10])
    exl.append(tct_values[11])
    fwl.append(tct_values[12])
    inl.append(tct_values[13])
    jjl.append(tct_values[14])
    jjrl.append(tct_values[15])
    jjsl.append(tct_values[16])
    lsl.append(tct_values[17])
    mdl.append(tct_values[18])
    nnl.append(tct_values[19])
    nnpl.append(tct_values[20])
    nnpsl.append(tct_values[21])
    nnsl.append(tct_values[22])
    pdtl.append(tct_values[23])
    posl.append(tct_values[24])
    prpl.append(tct_values[25])
    prpdl.append(tct_values[26])
    rbl.append(tct_values[27])
    rbrl.append(tct_values[28])
    rbsl.append(tct_values[29])
    rpl.append(tct_values[30])
    syml.append(tct_values[31])
    tol.append(tct_values[32])
    uhl.append(tct_values[33])
    vbl.append(tct_values[34])
    vbdl.append(tct_values[35])
    vbgl.append(tct_values[36])
    vbnl.append(tct_values[37])
    vbpl.append(tct_values[38])
    vbzl.append(tct_values[39])
    wdtl.append(tct_values[40])
    wpl.append(tct_values[41])
    wpdl.append(tct_values[42])
    wrbl.append(tct_values[43])
    oqml.append(tct_values[44])
    pl.append(tct_values[45])

In [16]:
#test_df_prepro = pd.get_dummies(test_df[1], columns=1)
test_df_prepro = test_df.loc[:, [1]]

In [17]:
test_df_prepro['$'] = dl
test_df_prepro["''"] = cqml
test_df_prepro['('] = opl
test_df_prepro[')'] = cpl
test_df_prepro[','] = cl
test_df_prepro['--'] = dal
test_df_prepro['.'] = stl
test_df_prepro[':'] = cel
test_df_prepro['CC'] = ccl
test_df_prepro['CD'] = cdl
test_df_prepro['DT'] = dtl
test_df_prepro['EX'] = exl
test_df_prepro['FW'] = fwl
test_df_prepro['IN'] = inl
test_df_prepro['JJ'] = jjl
test_df_prepro['JJR'] = jjrl
test_df_prepro['JJS'] = jjsl
test_df_prepro['LS'] = lsl
test_df_prepro['MD'] = mdl
test_df_prepro['NN'] = nnl
test_df_prepro['NNP'] = nnpl
test_df_prepro['NNPS'] = nnpsl
test_df_prepro['NNS'] = nnsl
test_df_prepro['PDT'] = pdtl
test_df_prepro['POS'] = posl
test_df_prepro['PRP'] = prpl
test_df_prepro['PRP$'] = prpdl
test_df_prepro['RB'] = rbl
test_df_prepro['RBR'] = rbrl
test_df_prepro['RBS'] = rbsl
test_df_prepro['RP'] = rpl
test_df_prepro['SYM'] = syml
test_df_prepro['TO'] = tol
test_df_prepro['UH'] = uhl
test_df_prepro['VB'] = vbl
test_df_prepro['VBD'] = vbdl
test_df_prepro['VBG'] = vbgl
test_df_prepro['VBN'] = vbnl
test_df_prepro['VBZ'] = vbzl
test_df_prepro['WDT'] = wdtl
test_df_prepro['WP'] = wpl
test_df_prepro['WP$'] = wpdl
test_df_prepro['WRB'] = wrbl
test_df_prepro['``'] = oqml
test_df_prepro['#'] = pl

test_df_prepro

,1,$,'',(,),",",--,.,:,CC,CD,DT,EX,FW,IN,JJ,JJR,JJS,LS,MD,NN,NNP,NNPS,NNS,PDT,POS,PRP,PRP$,RB,RBR,RBS,RP,SYM,TO,UH,VB,VBD,VBG,VBN,VBZ,WDT,WP,WP$,WRB,``,#
0,true,0,0,0,0,0,0,1,0,0,0,2,0,0,1,0,0,0,0,1,2,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
1,false,0,0,0,0,0,0,1,0,0,0,2,0,0,2,0,0,0,0,0,3,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0
2,false,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,2,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,2,0,0,0,0,0,0
3,half-true,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
4,pants-fire,0,0,0,0,1,0,2,0,0,0,3,0,0,4,1,0,0,0,0,4,3,0,2,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,half-true,0,0,0,0,0,0,1,0,0,0,1,0,0,2,0,0,1,0,0,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0
1263,barely-true,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1264,barely-true,0,0,0,0,1,0,1,0,0,1,2,0,0,2,3,0,0,0,0,2,4,0,2,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
1265,barely-true,0,0,0,0,0,0,1,0,1,1,1,0,0,2,1,0,0,0,0,2,3,0,1,0,0,1,1,0,0,0,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0


In [18]:
#target_labels = ['barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'true']
#X_test = test_df_prepro.drop(target_labels, axis=1)
#y_test = test_df_prepro[target_labels]

X_test = test_df_prepro.drop(1, axis=1)
y_test = test_df_prepro[1]

#### Train models

##### Different Classifiers

In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [22]:
from time import time
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", #"Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(2),
    SVC(kernel="linear", C=0.025, probability=True, random_state=0),
    SVC(gamma=2, C=1, probability=True, random_state=0),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5, random_state=0),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=0),
    MLPClassifier(alpha=1, max_iter=1000, random_state=0),
    AdaBoostClassifier(random_state=0),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

##### Muller loop

In [23]:
max_score = 0.0
max_class = ''
for name, clf in zip(names, classifiers):
    start_time = time()
    clf.fit(X_train, y_train)
    score = 100.0 * clf.score(X_test, y_test)
    print('Classifier = %s, Score (test, accuracy) = %.2f,' %(name, score), 'Training time = %.2f seconds' % (time() - start_time))
    
    if score > max_score:
        clf_best = clf
        max_score = score
        max_class = name

print(80*'-' )
print('Best --> Classifier = %s, Score (test, accuracy) = %.2f' %(max_class, max_score))

Classifier = Nearest Neighbors, Score (test, accuracy) = 20.52, Training time = 0.79 seconds
Classifier = Linear SVM, Score (test, accuracy) = 23.28, Training time = 57.37 seconds
Classifier = RBF SVM, Score (test, accuracy) = 20.92, Training time = 96.19 seconds
Classifier = Decision Tree, Score (test, accuracy) = 22.18, Training time = 0.04 seconds
Classifier = Random Forest, Score (test, accuracy) = 23.36, Training time = 0.06 seconds
Classifier = Neural Net, Score (test, accuracy) = 22.81, Training time = 8.28 seconds
Classifier = AdaBoost, Score (test, accuracy) = 22.42, Training time = 1.35 seconds
Classifier = Naive Bayes, Score (test, accuracy) = 8.37, Training time = 0.03 seconds
Classifier = QDA, Score (test, accuracy) = 9.39, Training time = 0.05 seconds
--------------------------------------------------------------------------------
Best --> Classifier = Random Forest, Score (test, accuracy) = 23.36


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


##### Feature importance

In [24]:
pd.Series(classifiers[4].feature_importances_,X_train.columns).sort_values(ascending=False)

CD      0.076583
NNP     0.073825
.       0.055862
JJS     0.051360
JJR     0.051244
DT      0.050714
VB      0.050250
NN      0.045695
IN      0.037982
PRP     0.037143
:       0.037037
,       0.035806
VBG     0.034089
VBN     0.024891
VBD     0.024385
TO      0.022101
VBZ     0.021748
RB      0.020260
(       0.019218
''      0.018700
``      0.018299
JJ      0.017645
PRP$    0.016300
NNS     0.015065
FW      0.014020
WDT     0.013284
WRB     0.012958
NNPS    0.012538
MD      0.010477
)       0.010310
$       0.009559
RBR     0.009182
#       0.008139
RP      0.007832
EX      0.007163
POS     0.007104
WP      0.006578
PDT     0.005626
CC      0.005072
WP$     0.003959
LS      0.000000
RBS     0.000000
--      0.000000
SYM     0.000000
UH      0.000000
dtype: float64

### Data preparation (POS percentage)

#### Train data

##### Calculate the percentage and create dataset

In [29]:
dl = []
cqml = []
opl = []
cpl = []
cl = []
dal = []
stl = []
cel = []
ccl = []
cdl = []
dtl = []
exl = []
fwl = []
inl = []
jjl = []
jjrl = []
jjsl = []
lsl = []
mdl = []
nnl = []
nnpl = []
nnpsl = []
nnsl = []
pdtl = []
posl = []
prpl = []
prpdl = []
rbl = []
rbrl = []
rbsl = []
rpl = []
syml = []
tol = []
uhl = []
vbl = []
vbdl = []
vbgl = []
vbnl = []
vbpl = []
vbzl = []
wdtl = []
wpl = []
wpdl = []
wrbl = []
oqml = []
pl = []

for tct in train_tag_counters:
    tct_values = list(tct.values())
    total = sum(tct_values)

    dl.append(tct_values[0]/total)
    cqml.append(tct_values[1]/total)
    opl.append(tct_values[2]/total)
    cpl.append(tct_values[3]/total)
    cl.append(tct_values[4]/total)
    dal.append(tct_values[5]/total)
    stl.append(tct_values[6]/total)
    cel.append(tct_values[7]/total)
    ccl.append(tct_values[8]/total)
    cdl.append(tct_values[9]/total)
    dtl.append(tct_values[10]/total)
    exl.append(tct_values[11]/total)
    fwl.append(tct_values[12]/total)
    inl.append(tct_values[13]/total)
    jjl.append(tct_values[14]/total)
    jjrl.append(tct_values[15]/total)
    jjsl.append(tct_values[16]/total)
    lsl.append(tct_values[17]/total)
    mdl.append(tct_values[18]/total)
    nnl.append(tct_values[19]/total)
    nnpl.append(tct_values[20]/total)
    nnpsl.append(tct_values[21]/total)
    nnsl.append(tct_values[22]/total)
    pdtl.append(tct_values[23]/total)
    posl.append(tct_values[24]/total)
    prpl.append(tct_values[25]/total)
    prpdl.append(tct_values[26]/total)
    rbl.append(tct_values[27]/total)
    rbrl.append(tct_values[28]/total)
    rbsl.append(tct_values[29]/total)
    rpl.append(tct_values[30]/total)
    syml.append(tct_values[31]/total)
    tol.append(tct_values[32]/total)
    uhl.append(tct_values[33]/total)
    vbl.append(tct_values[34]/total)
    vbdl.append(tct_values[35]/total)
    vbgl.append(tct_values[36]/total)
    vbnl.append(tct_values[37]/total)
    vbpl.append(tct_values[38]/total)
    vbzl.append(tct_values[39]/total)
    wdtl.append(tct_values[40]/total)
    wpl.append(tct_values[41]/total)
    wpdl.append(tct_values[42]/total)
    wrbl.append(tct_values[43]/total)
    oqml.append(tct_values[44]/total)
    pl.append(tct_values[45]/total)

In [30]:
#train_df_prepro = pd.get_dummies(train_df[1], columns=1)
train_df_prepro = train_df.loc[:, [1]]

In [31]:
train_df_prepro['$'] = dl
train_df_prepro["''"] = cqml
train_df_prepro['('] = opl
train_df_prepro[')'] = cpl
train_df_prepro[','] = cl
train_df_prepro['--'] = dal
train_df_prepro['.'] = stl
train_df_prepro[':'] = cel
train_df_prepro['CC'] = ccl
train_df_prepro['CD'] = cdl
train_df_prepro['DT'] = dtl
train_df_prepro['EX'] = exl
train_df_prepro['FW'] = fwl
train_df_prepro['IN'] = inl
train_df_prepro['JJ'] = jjl
train_df_prepro['JJR'] = jjrl
train_df_prepro['JJS'] = jjsl
train_df_prepro['LS'] = lsl
train_df_prepro['MD'] = mdl
train_df_prepro['NN'] = nnl
train_df_prepro['NNP'] = nnpl
train_df_prepro['NNPS'] = nnpsl
train_df_prepro['NNS'] = nnsl
train_df_prepro['PDT'] = pdtl
train_df_prepro['POS'] = posl
train_df_prepro['PRP'] = prpl
train_df_prepro['PRP$'] = prpdl
train_df_prepro['RB'] = rbl
train_df_prepro['RBR'] = rbrl
train_df_prepro['RBS'] = rbsl
train_df_prepro['RP'] = rpl
train_df_prepro['SYM'] = syml
train_df_prepro['TO'] = tol
train_df_prepro['UH'] = uhl
train_df_prepro['VB'] = vbl
train_df_prepro['VBD'] = vbdl
train_df_prepro['VBG'] = vbgl
train_df_prepro['VBN'] = vbnl
train_df_prepro['VBZ'] = vbzl
train_df_prepro['WDT'] = wdtl
train_df_prepro['WP'] = wpl
train_df_prepro['WP$'] = wpdl
train_df_prepro['WRB'] = wrbl
train_df_prepro['``'] = oqml
train_df_prepro['#'] = pl

train_df_prepro

,1,$,'',(,),",",--,.,:,CC,CD,DT,EX,FW,IN,JJ,JJR,JJS,LS,MD,NN,NNP,NNPS,NNS,PDT,POS,PRP,PRP$,RB,RBR,RBS,RP,SYM,TO,UH,VB,VBD,VBG,VBN,VBZ,WDT,WP,WP$,WRB,``,#
0,false,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.083333,0.0,0.000000,0.0,0.083333,0.000000,0.000000,0.083333,0.083333,0.000000,0.0,0.0,0.000000,0.250000,0.083333,0.083333,0.083333,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,half-true,0.0,0.000000,0.035714,0.035714,0.000000,0.0,0.071429,0.0,0.000000,0.0,0.035714,0.000000,0.000000,0.107143,0.035714,0.000000,0.0,0.0,0.000000,0.178571,0.142857,0.000000,0.000000,0.0,0.0,0.035714,0.000000,0.000000,0.0,0.0,0.035714,0.0,0.035714,0.0,0.035714,0.142857,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.071429,0.000000,0.0
2,mostly-true,0.0,0.045455,0.000000,0.000000,0.000000,0.0,0.045455,0.0,0.000000,0.0,0.090909,0.000000,0.000000,0.181818,0.000000,0.000000,0.0,0.0,0.000000,0.090909,0.318182,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.045455,0.0,0.045455,0.000000,0.045455,0.000000,0.045455,0.000000,0.0,0.0,0.000000,0.045455,0.0
3,false,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.076923,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.0,0.0,0.000000,0.384615,0.076923,0.000000,0.076923,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.076923,0.0,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,half-true,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.090909,0.0,0.000000,0.0,0.181818,0.000000,0.000000,0.181818,0.090909,0.000000,0.0,0.0,0.000000,0.272727,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.090909,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,mostly-true,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.000000,0.0,0.055556,0.111111,0.000000,0.222222,0.000000,0.055556,0.0,0.0,0.000000,0.222222,0.055556,0.000000,0.111111,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
10236,mostly-true,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.058824,0.0,0.058824,0.0,0.117647,0.000000,0.000000,0.117647,0.000000,0.000000,0.0,0.0,0.000000,0.117647,0.235294,0.058824,0.058824,0.0,0.0,0.000000,0.000000,0.058824,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
10237,half-true,0.0,0.000000,0.000000,0.000000,0.064516,0.0,0.032258,0.0,0.000000,0.0,0.064516,0.000000,0.000000,0.161290,0.032258,0.032258,0.0,0.0,0.032258,0.096774,0.225806,0.000000,0.032258,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.032258,0.0,0.032258,0.000000,0.000000,0.032258,0.096774,0.032258,0.0,0.0,0.000000,0.000000,0.0
10238,false,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.083333,0.0,0.083333,0.0,0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.0,0.0,0.000000,0.166667,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.083333,0.0,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [36]:
#target_labels = ['barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'true']
#X_train = train_df_prepro.drop(target_labels, axis=1)
#y_train = train_df_prepro[target_labels]

X_train = train_df_prepro.drop(1, axis=1)
y_train = train_df_prepro[1]

#### Test data

##### Calculate the percentage and create dataset

In [33]:
dl = []
cqml = []
opl = []
cpl = []
cl = []
dal = []
stl = []
cel = []
ccl = []
cdl = []
dtl = []
exl = []
fwl = []
inl = []
jjl = []
jjrl = []
jjsl = []
lsl = []
mdl = []
nnl = []
nnpl = []
nnpsl = []
nnsl = []
pdtl = []
posl = []
prpl = []
prpdl = []
rbl = []
rbrl = []
rbsl = []
rpl = []
syml = []
tol = []
uhl = []
vbl = []
vbdl = []
vbgl = []
vbnl = []
vbpl = []
vbzl = []
wdtl = []
wpl = []
wpdl = []
wrbl = []
oqml = []
pl = []

for tct in test_tag_counters:
    tct_values = list(tct.values())
    total = sum(tct_values)

    dl.append(tct_values[0]/total)
    cqml.append(tct_values[1]/total)
    opl.append(tct_values[2]/total)
    cpl.append(tct_values[3]/total)
    cl.append(tct_values[4]/total)
    dal.append(tct_values[5]/total)
    stl.append(tct_values[6]/total)
    cel.append(tct_values[7]/total)
    ccl.append(tct_values[8]/total)
    cdl.append(tct_values[9]/total)
    dtl.append(tct_values[10]/total)
    exl.append(tct_values[11]/total)
    fwl.append(tct_values[12]/total)
    inl.append(tct_values[13]/total)
    jjl.append(tct_values[14]/total)
    jjrl.append(tct_values[15]/total)
    jjsl.append(tct_values[16]/total)
    lsl.append(tct_values[17]/total)
    mdl.append(tct_values[18]/total)
    nnl.append(tct_values[19]/total)
    nnpl.append(tct_values[20]/total)
    nnpsl.append(tct_values[21]/total)
    nnsl.append(tct_values[22]/total)
    pdtl.append(tct_values[23]/total)
    posl.append(tct_values[24]/total)
    prpl.append(tct_values[25]/total)
    prpdl.append(tct_values[26]/total)
    rbl.append(tct_values[27]/total)
    rbrl.append(tct_values[28]/total)
    rbsl.append(tct_values[29]/total)
    rpl.append(tct_values[30]/total)
    syml.append(tct_values[31]/total)
    tol.append(tct_values[32]/total)
    uhl.append(tct_values[33]/total)
    vbl.append(tct_values[34]/total)
    vbdl.append(tct_values[35]/total)
    vbgl.append(tct_values[36]/total)
    vbnl.append(tct_values[37]/total)
    vbpl.append(tct_values[38]/total)
    vbzl.append(tct_values[39]/total)
    wdtl.append(tct_values[40]/total)
    wpl.append(tct_values[41]/total)
    wpdl.append(tct_values[42]/total)
    wrbl.append(tct_values[43]/total)
    oqml.append(tct_values[44]/total)
    pl.append(tct_values[45]/total)

In [34]:
#test_df_prepro = pd.get_dummies(test_df[1], columns=1)
test_df_prepro = test_df.loc[:, [1]]

In [35]:
test_df_prepro['$'] = dl
test_df_prepro["''"] = cqml
test_df_prepro['('] = opl
test_df_prepro[')'] = cpl
test_df_prepro[','] = cl
test_df_prepro['--'] = dal
test_df_prepro['.'] = stl
test_df_prepro[':'] = cel
test_df_prepro['CC'] = ccl
test_df_prepro['CD'] = cdl
test_df_prepro['DT'] = dtl
test_df_prepro['EX'] = exl
test_df_prepro['FW'] = fwl
test_df_prepro['IN'] = inl
test_df_prepro['JJ'] = jjl
test_df_prepro['JJR'] = jjrl
test_df_prepro['JJS'] = jjsl
test_df_prepro['LS'] = lsl
test_df_prepro['MD'] = mdl
test_df_prepro['NN'] = nnl
test_df_prepro['NNP'] = nnpl
test_df_prepro['NNPS'] = nnpsl
test_df_prepro['NNS'] = nnsl
test_df_prepro['PDT'] = pdtl
test_df_prepro['POS'] = posl
test_df_prepro['PRP'] = prpl
test_df_prepro['PRP$'] = prpdl
test_df_prepro['RB'] = rbl
test_df_prepro['RBR'] = rbrl
test_df_prepro['RBS'] = rbsl
test_df_prepro['RP'] = rpl
test_df_prepro['SYM'] = syml
test_df_prepro['TO'] = tol
test_df_prepro['UH'] = uhl
test_df_prepro['VB'] = vbl
test_df_prepro['VBD'] = vbdl
test_df_prepro['VBG'] = vbgl
test_df_prepro['VBN'] = vbnl
test_df_prepro['VBZ'] = vbzl
test_df_prepro['WDT'] = wdtl
test_df_prepro['WP'] = wpl
test_df_prepro['WP$'] = wpdl
test_df_prepro['WRB'] = wrbl
test_df_prepro['``'] = oqml
test_df_prepro['#'] = pl

test_df_prepro

,1,$,'',(,),",",--,.,:,CC,CD,DT,EX,FW,IN,JJ,JJR,JJS,LS,MD,NN,NNP,NNPS,NNS,PDT,POS,PRP,PRP$,RB,RBR,RBS,RP,SYM,TO,UH,VB,VBD,VBG,VBN,VBZ,WDT,WP,WP$,WRB,``,#
0,true,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.000000,0.000000,0.166667,0.0,0.0,0.083333,0.000000,0.000000,0.000000,0.0,0.083333,0.166667,0.083333,0.0,0.083333,0.0,0.0,0.000000,0.000000,0.083333,0.0,0.0,0.0,0.0,0.000000,0.000000,0.083333,0.000000,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
1,false,0.0,0.0,0.0,0.0,0.000000,0.0,0.076923,0.0,0.000000,0.000000,0.153846,0.0,0.0,0.153846,0.000000,0.000000,0.000000,0.0,0.000000,0.230769,0.076923,0.0,0.076923,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.076923,0.000000,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.0,0.0,0.000000,0.0,0.0
2,false,0.0,0.0,0.0,0.0,0.000000,0.0,0.090909,0.0,0.000000,0.000000,0.090909,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.090909,0.181818,0.0,0.090909,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.090909,0.000000,0.090909,0.000000,0.000000,0.090909,0.181818,0.000000,0.0,0.0,0.000000,0.0,0.0
3,half-true,0.0,0.0,0.0,0.0,0.000000,0.0,0.071429,0.0,0.000000,0.000000,0.071429,0.0,0.0,0.071429,0.000000,0.000000,0.000000,0.0,0.071429,0.142857,0.285714,0.0,0.071429,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.071429,0.0,0.0,0.000000,0.0,0.0
4,pants-fire,0.0,0.0,0.0,0.0,0.038462,0.0,0.076923,0.0,0.000000,0.000000,0.115385,0.0,0.0,0.153846,0.038462,0.000000,0.000000,0.0,0.000000,0.153846,0.115385,0.0,0.076923,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.038462,0.038462,0.038462,0.038462,0.000000,0.038462,0.000000,0.000000,0.0,0.0,0.038462,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,half-true,0.0,0.0,0.0,0.0,0.000000,0.0,0.071429,0.0,0.000000,0.000000,0.071429,0.0,0.0,0.142857,0.000000,0.000000,0.071429,0.0,0.000000,0.428571,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.0,0.000000,0.0,0.0
1263,barely-true,0.0,0.0,0.0,0.0,0.000000,0.0,0.142857,0.0,0.000000,0.000000,0.142857,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.142857,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.285714,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
1264,barely-true,0.0,0.0,0.0,0.0,0.043478,0.0,0.043478,0.0,0.000000,0.043478,0.086957,0.0,0.0,0.086957,0.130435,0.000000,0.000000,0.0,0.000000,0.086957,0.173913,0.0,0.086957,0.0,0.0,0.000000,0.000000,0.043478,0.0,0.0,0.0,0.0,0.043478,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
1265,barely-true,0.0,0.0,0.0,0.0,0.000000,0.0,0.052632,0.0,0.052632,0.052632,0.052632,0.0,0.0,0.105263,0.052632,0.000000,0.000000,0.0,0.000000,0.105263,0.157895,0.0,0.052632,0.0,0.0,0.052632,0.052632,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.105263,0.000000,0.052632,0.052632,0.000000,0.0,0.0,0.000000,0.0,0.0


In [37]:
#target_labels = ['barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'true']
#X_test = test_df_prepro.drop(target_labels, axis=1)
#y_test = test_df_prepro[target_labels]

X_test = test_df_prepro.drop(1, axis=1)
y_test = test_df_prepro[1]

#### Train models

##### Different Classifiers

In [38]:
classifiers = [
    KNeighborsClassifier(2),
    SVC(kernel="linear", C=0.025, probability=True, random_state=0),
    SVC(gamma=2, C=1, probability=True, random_state=0),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5, random_state=0),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=0),
    MLPClassifier(alpha=1, max_iter=1000, random_state=0),
    AdaBoostClassifier(random_state=0),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

##### Muller loop

In [39]:
max_score = 0.0
max_class = ''
for name, clf in zip(names, classifiers):
    start_time = time()
    clf.fit(X_train, y_train)
    score = 100.0 * clf.score(X_test, y_test)
    print('Classifier = %s, Score (test, accuracy) = %.2f,' %(name, score), 'Training time = %.2f seconds' % (time() - start_time))
    
    if score > max_score:
        clf_best_percData = clf
        max_score = score
        max_class = name

print(80*'-' )
print('Best --> Classifier = %s, Score (test, accuracy) = %.2f' %(max_class, max_score))

Classifier = Nearest Neighbors, Score (test, accuracy) = 21.70, Training time = 0.76 seconds
Classifier = Linear SVM, Score (test, accuracy) = 20.92, Training time = 52.90 seconds
Classifier = RBF SVM, Score (test, accuracy) = 22.89, Training time = 77.30 seconds
Classifier = Decision Tree, Score (test, accuracy) = 22.89, Training time = 0.07 seconds
Classifier = Random Forest, Score (test, accuracy) = 22.73, Training time = 0.06 seconds
Classifier = Neural Net, Score (test, accuracy) = 20.68, Training time = 2.18 seconds
Classifier = AdaBoost, Score (test, accuracy) = 21.78, Training time = 1.52 seconds
Classifier = Naive Bayes, Score (test, accuracy) = 9.16, Training time = 0.03 seconds
Classifier = QDA, Score (test, accuracy) = 7.97, Training time = 0.05 seconds
--------------------------------------------------------------------------------
Best --> Classifier = RBF SVM, Score (test, accuracy) = 22.89


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


##### Feature importance

In [40]:
pd.Series(classifiers[4].feature_importances_,X_train.columns).sort_values(ascending=False)

NNP     0.102293
CD      0.060676
DT      0.055968
JJS     0.055581
VB      0.049938
.       0.049739
VBZ     0.045048
JJR     0.042371
NN      0.040971
VBD     0.035094
PRP     0.032539
VBN     0.031637
NNS     0.030687
,       0.029900
IN      0.028426
TO      0.027038
JJ      0.026667
$       0.025662
WRB     0.025138
VBG     0.018760
WP      0.018231
:       0.016242
EX      0.015664
PRP$    0.014373
``      0.013002
(       0.011720
RB      0.010921
PDT     0.009670
FW      0.008723
WDT     0.008225
NNPS    0.008166
''      0.008141
MD      0.007973
#       0.006732
)       0.006667
WP$     0.004665
CC      0.004082
UH      0.003421
RP      0.002586
RBS     0.002454
POS     0.002261
RBR     0.001948
SYM     0.000000
--      0.000000
LS      0.000000
dtype: float64

##### Conclusion
Most models deteriorated..

## TO-DO

*   Combine some POS
*   Find micro-factors
*   Assign weights in a polynomial equation



## Reference

http://www.nltk.org/book/ch05.html
